Hello<br>
Welcome to more chaos

In [19]:
import pandas as pd
from sentence_transformers import SentenceTransformer
# ^^ hugging face library for word embeddings
import sqlite3
import re
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import random
from keybert import KeyBERT

print("howdy")

howdy


Ok so first I'm going to just pull the data for seeing the posts with tags that the user has liked most. <br>
We're also going to come up with a query for the random user to see what tags they like <br>
Honestly we can't <i>really</i> test this data with our dummy data, so we kind of just have to trust.

In [20]:
# Pull random user

user = random.randint(0,500)

liked_posts_query = \
f"SELECT posts.tags \
FROM likes \
JOIN posts on likes.post_id = posts.id \
WHERE likes.user_id = {user}"

user_query = \
f"SELECT * FROM users WHERE id={user}"

print(f"User ID populated: {user}")

User ID populated: 206


In [21]:
con = sqlite3.connect("/home/acequantum/playtime/dummy_data/1_dummy_data.db")

df_user = pd.read_sql_query(user_query, con)

print(df_user)

df_liked_posts = pd.read_sql_query(liked_posts_query, con)

print(df_liked_posts)

con.close()

    id           name               email  \
0  206  Angelica Carr  lisa29@example.net   

                                                bios  
0  Situation discussion cell talk address but som...  
                                                 tags
0                    perfumesColognes, antiqueJewelry
1                       scooterParts, rareManuscripts
2                  customStickers, musicalInstruments
3      sewingMachines, scooterParts, designerHandbags
4                                 mechanicalKeyboards
5                   gourmetFoodItems, artisanalCheese
6               textbooks, hikingGear, embroideryKits
7                paintballEquipment, gourmetFoodItems
8                                           tailoring
9                                          VRHeadsets
10              tutoring, bikeRepairs, customClothing
11         handmadeJewelry, tutoring, collectiblePins
12               collectiblePins, rareCoinsOrCurrency
13                     retroToys, vintageVi

In [22]:
def camel_case_split(str):
    return re.sub(r'([a-z])([A-Z])',
                      r'\1 \2', str)

df_liked_posts['tags'] = df_liked_posts['tags'].apply(camel_case_split)

print(df_liked_posts)


                                                 tags
0                  perfumes Colognes, antique Jewelry
1                     scooter Parts, rare Manuscripts
2                custom Stickers, musical Instruments
3   sewing Machines, scooter Parts, designer Handbags
4                                mechanical Keyboards
5                gourmet Food Items, artisanal Cheese
6             textbooks, hiking Gear, embroidery Kits
7             paintball Equipment, gourmet Food Items
8                                           tailoring
9                                          VRHeadsets
10            tutoring, bike Repairs, custom Clothing
11       handmade Jewelry, tutoring, collectible Pins
12           collectible Pins, rare Coins Or Currency
13                  retro Toys, vintage Vinyl Records
14                                              bread
15                                    embroidery Kits
16    fitness Trackers, graphic Design, trading Cards
17                          

In [24]:
df_liked_posts['new_tags'] = df_liked_posts['tags'].str.split(', ')

all_tags = df_liked_posts['new_tags'].explode()

print(f"all tags: {all_tags.tolist()}")

all tags: ['perfumes Colognes', 'antique Jewelry', 'scooter Parts', 'rare Manuscripts', 'custom Stickers', 'musical Instruments', 'sewing Machines', 'scooter Parts', 'designer Handbags', 'mechanical Keyboards', 'gourmet Food Items', 'artisanal Cheese', 'textbooks', 'hiking Gear', 'embroidery Kits', 'paintball Equipment', 'gourmet Food Items', 'tailoring', 'VRHeadsets', 'tutoring', 'bike Repairs', 'custom Clothing', 'handmade Jewelry', 'tutoring', 'collectible Pins', 'collectible Pins', 'rare Coins Or Currency', 'retro Toys', 'vintage Vinyl Records', 'bread', 'embroidery Kits', 'fitness Trackers', 'graphic Design', 'trading Cards', 'craft Supplies', 'rare Board Games', 'vintage Lamps', 'collectible Pins', 'horror Memorabilia', 'pet Sitting Dog Walking', 'recording Equipment', 'embroidery Kits', 'rare Board Games', 'fashion Accessories', 'voice Acting', 'custom Art', 'vintage Postcards', 'tattoo Designs', 'graphic Design', 'books', 'autographed Memorabilia', 'pet Sitting Dog Walking', 'a

In [29]:
tag_counts = all_tags.value_counts().sort_values(ascending=False).reset_index()

tag_counts.columns = ['tag', 'count']
print(tag_counts)
# print(f"total liked tags: {}")
print(type(tag_counts))
total = tag_counts['count'].sum()
print(total)

                        tag  count
0          rare Board Games      4
1        gourmet Food Items      3
2           embroidery Kits      3
3                     books      3
4   autographed Memorabilia      3
..                      ...    ...
62             fishing Gear      1
63            fresh Produce      1
64                local Art      1
65               sunglasses      1
66              houseplants      1

[67 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
101


In [33]:
# Tbh these embeddings won't really make sense but we'll do what we can.
# I also need to just straight up grab stuff I guess.
# I guess I'll start with the weighted die roll.
# I have no idea how to do that.

random_sample = tag_counts.sample(n=3, weights='count')['tag']

print(random_sample)

33    mechanical Keyboards
6         artisanal Cheese
63           fresh Produce
Name: tag, dtype: object


In [34]:
# Ok so we have a random sampling of tags, 
# let's grab 3 random posts with those tags,
# And return the ones that have the most likes
con = sqlite3.connect("/home/acequantum/playtime/dummy_data/1_dummy_data.db")

df_all_posts = pd.read_sql_query("SELECT * FROM posts;", con)

df_all_posts['new_tags'] = df_all_posts['tags']

df_all_posts['new_tags'] = df_all_posts['tags'].apply(camel_case_split)

print(df_all_posts.head())

con.close()

   id  user_id trade_status  \
0   1        1         None   
1   2        1         None   
2   3        2         None   
3   4        2         None   
4   5        2         None   

                                                tags  \
0     mangaAnimeMerch, diyTools, mechanicalKeyboards   
1  collectiblePins, petSittingDogWalking, vintage...   
2                             customToys, bakedGoods   
3     clayModelingKits, codingLessons, tattooDesigns   
4                 leatherGoods, bikeRepairs, watches   

                                                text  \
0  Really describe particularly whose opportunity...   
1  Analysis father board everybody. Car begin evi...   
2  Per assume community dog happy really stop. Ch...   
3  By song new know. Particularly third all inves...   
4  Three writer policy building. Offer the capita...   

                                            new_tags  
0  manga Anime Merch, diy Tools, mechanical Keybo...  
1  collectible Pins, pet Sitt